In [1]:
import pandas as pd
import scipy.io
import re
import sys
import warnings
import pickle

In [2]:
from mvpa2.datasets.mri import fmri_dataset

Failed to import duecredit due to No module named 'duecredit'
/home/bsmith16/.conda/envs/py3_mvpa/lib/python3.8/site-packages/mvpa2/datasets/base.py:465: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def __init__(self, shape=None, sid=None, fid=None, dtype=np.float):


In [3]:
class BehavioralDataNotFoundForBrainDataException(Exception):
    """Behavioral data could not be matched to a subject."""
    pass

Replicating earlier work on mvpa. Try not to overly complicate it--the main point is just to verify we get similar results on a different package to validate prior work. ANd we are primarily interested in validating the very high cross-validation results I got with nltools. Should aim for readable code.

In [4]:
from mvpa2 import *

In [5]:


nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"

In [6]:
brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_conditions_84subs_correct_cond.pkl'
results_filepath=ml_data_folderpath + "/SST/mvpa_tt_res_v3_conditions_84subs_twoclasses_pfcmask.pkl"

In [7]:
include_exclude_list = pd.read_csv("../nsc_subject_exclusions.csv")

In [8]:
test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20211027T173724.csv")
test_train_df_raw = test_train_df_raw.merge(include_exclude_list[include_exclude_list.Task=='SST'],left_on='sub_label',right_on='SubjectId',how='left')
test_train_df_raw.loc[test_train_df_raw.Include.isna(),'Include'] = True
test_train_df = test_train_df_raw[test_train_df_raw.Include==True]
exclude_subjects = ['DEV061','DEV185','DEV187','DEV189','DEV190','DEV192','DEV198','DEV203','DEV220','DEV221']
train_subjs = test_train_df.loc[test_train_df.SplitGroup=='Train','sub_label'].tolist()#only get the train subjects; ignore those previously marked hold-out

In [9]:
train_subjs_selected = [ts for ts in train_subjs if (ts not in exclude_subjects)]

In [11]:
", ".join(train_subjs_selected)

'DEV005, DEV006, DEV009, DEV010, DEV012, DEV013, DEV014, DEV015, DEV016, DEV017, DEV018, DEV019, DEV021, DEV022, DEV023, DEV024, DEV025, DEV026, DEV027, DEV028, DEV029, DEV030, DEV034, DEV035, DEV036, DEV039, DEV040, DEV041, DEV042, DEV043, DEV046, DEV048, DEV049, DEV050, DEV051, DEV052, DEV053, DEV055, DEV056, DEV057, DEV058, DEV059, DEV060, DEV062, DEV068, DEV069, DEV071, DEV073, DEV074, DEV076, DEV077, DEV079, DEV083, DEV084, DEV085, DEV086, DEV087, DEV089, DEV011, DEV020, DEV047, DEV064, DEV067, DEV097, DEV105, DEV107, DEV135, DEV141, DEV145, DEV157, DEV158, DEV159, DEV164, DEV169, DEV171, DEV173, DEV177, DEV186, DEV197, DEV206, DEV215, DEV216, DEV217, DEV218'

In [10]:
individual_differences = pd.read_csv(ml_data_folderpath + "/data_by_ppt.csv")
individual_differences = individual_differences.rename(columns={'SID':'subject'})
individual_differences['wave']=1

In [11]:
import sys

sys.path.append(os.path.abspath("../../ml/"))

We probably actually want to start the pipeline from the betas rather than loading from pickle. to be continued...

In [12]:
from mvpa_pipeline_utils import get_Brain_Data_betas_as_mvpa_for_sub, import_beta_series_pymvpa2, sa_to_df

/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/mvpa_pipeline_utils.py:69: DeprecationWarning: invalid escape sequence \(
  condition_label = re.search('Sn\(1\)\s(.*)\*bf\(1\)',b_description)


In [13]:
import mvpa2.datasets 

In [14]:
my_test_data =import_beta_series_pymvpa2(
    train_subjs_selected[0:2],'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_cond_',
    beta_processing_args = {
        'contrast_condition_1':'CorrectStop',
        'contrast_condition_2':'CorrectGo',
        'spatially_concatenate':False
    }
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV005/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV005/beta_0001.nii
853707, 228328, WARNING: Detected incorrect (nan) scl_ fields. Resetting to scl_slope=1.0 and scl_inter=0.0
 * Please note: warnings are printed only once, but underlying problem might occur many times *
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV005     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV005     1  
<SampleAttributesC

In [15]:
sa_to_df(my_test_data.sa)

,time_indices,time_coords,condition_index,condition_label,raw_beta_description,beta,subject,wave,cancer_promoting_minus_preventing_FCI,cancer_promoting_minus_preventing_FFQ,...,WTP_unhealthy,WTP_unhealthy_minus_healthy,SRHI_healthy_minus_unhealthy,RTFS_f1_minus_f2,IPAQ_walkingMETminutes,IPAQ_moderateMETminutes,IPAQ_vigorousMETminutes,IPAQ_total_METminutes,IPAQ_MET_kCal,birthsex_factor
0,0,0.0,"0, 1","CorrectGo, CorrectStop","spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1), s...","beta_0001.nii, beta_0002.nii",DEV005,1,0.083871,0.321154,...,2.451613,0.170363,-38.0,NaN,99.0,160.0,NaN,NaN,NaN,Female
1,0,0.0,"0, 1","CorrectGo, CorrectStop","spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1), s...","beta_0001.nii, beta_0002.nii",DEV006,1,0.430645,-0.603846,...,1.838710,-0.494624,-16.0,NaN,1039.5,1080.0,NaN,NaN,NaN,Female


In [ ]:
my_test_data =import_beta_series_pymvpa2(
    train_subjs_selected[0:10],'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_cond',
    concatenate_condition_labels=False
)

In [76]:
my_test_data =import_beta_series_pymvpa2(
    train_subjs_selected,'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_cond',
    concatenate_condition_labels=False
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV005
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV005/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV005/beta_0001.nii
853707, 228328, 315425
DEV006
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV006/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV006/beta_0001.nii
853707, 228328, 147551
DEV009
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV009/SPM.mat
not enough conditions for subject DEV009. Skipping this subject.
DEV010
/gpfs/proje

853707, 228328, 277397
DEV034
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV034/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV034/beta_0001.nii
853707, 228328, 179592
DEV035
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV035/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV035/beta_0001.nii
853707, 228328, 329650
DEV036
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV036/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/san

853707, 228328, 208997
DEV064
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV064/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV064/beta_0001.nii
853707, 228328, 245724
DEV067
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV067/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV067/beta_0001.nii
853707, 228328, 274940
DEV068
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV068/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/san

853707, 228328, 228334
DEV158
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV158/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV158/beta_0001.nii
853707, 228328, 206980
DEV159
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV159/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV159/beta_0001.nii
853707, 228328, 320310
DEV164
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV164/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/san

In [3]:
my_test_data =import_beta_series_pymvpa2(
    train_subjs_selected,'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_cor_stop_minus_go_contrast',
    beta_processing_args = {
        'contrast_condition_1':'CorrectStop',
        'contrast_condition_2':'CorrectGo',
        'spatially_concatenate':False
    }
)

NameError: name 'train_subjs_selected' is not defined

In [110]:
my_test_data.a['get_sa_as_df'] = lambda: sa_to_df(my_test_data.sa)

In [2]:
import sys
import os
sys.path.append(os.path.abspath("../../ml/"))

import pandas as pd
import scipy.io
import re
import sys
import warnings
import pickle
from mvpa2.datasets.mri import fmri_dataset
from mvpa2 import *
import mvpa2.datasets 
import nibabel as nib
import numpy as np
import nibabel as nib
from nilearn.image import resample_img

class BehavioralDataNotFoundForBrainDataException(Exception):
    """Behavioral data could not be matched to a subject."""
    pass

def sa_to_df(sa_collectable,column_filter=None):
    sa_index = pd.DataFrame({sa:sa_collectable[sa].value for sa in sa_collectable if (column_filter is None) or (sa in column_filter)})
    return(sa_index)
        
## get a list of the subject folders
## iterate through them
## pass in a design matrix appropriate for that subject (pretty much just subject ID and then name of condition)
## append

def import_beta_series_pymvpa2(subjs,first_level_fileid,out_folder = '../data/', 
                                    conditions_to_include=None,
                                    condition_count_required=None,
                                    supplementary_df = None,
                                         out_file_suffix ='',
                                    #concatenate_condition_labels=False,
                               #mask = "beta",
                               #masking_threshold=0,
                               beta_processing_args={}
                                   ):
    ## get a list of the subject folders
    sst_wt_repo = '/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/'
    first_level_path = sst_wt_repo + first_level_fileid + "/"
    print(first_level_path)
    subj_count = len(subjs)
    ## iterate through them
    subjs.sort()
    bd_dict={}
    #get the brain data from the beta files
    for sl in subjs:
        print(sl)
        
        #get the design data
        
        #load the matrix associated with this file
        print(first_level_path + 'sub-' + sl +'/SPM.mat')
        sl_mat = scipy.io.loadmat(
            first_level_path + 'sub-' + sl +'/SPM.mat',
            simplify_cells=True            
        )

        #convert from an SPM.mat file into a dataframe list of the betas
        beta_dict_list = []
        mat_betas = sl_mat['SPM']['Vbeta']
        for beta_i in range(len(mat_betas)):
            beta = mat_betas[beta_i]
            b_description = beta['descrip']
            condition_label = re.search('Sn\(1\)\s(.*)\*bf\(1\)',b_description)
            if (condition_label is not None):
                beta_dict_list = beta_dict_list + [{
                    'condition_index':beta_i,
                    'condition_label':condition_label.group(1),
                    'raw_beta_description':beta['descrip'],
                    'beta':beta['fname']
                }]
        #    print(condition_label)
        beta_df = pd.DataFrame(beta_dict_list)

        #fill in the details related to the subject
        beta_df['subject']=sl
        beta_df['wave']=1
        
        if conditions_to_include is not None:
            #cut down conditions to just those specified
            #also ensures that the beta_df is in the ORDER SPECIFIED in conditions_to_include
            #beta_df = beta_df.loc[beta_df.condition_label.isin(conditions_to_include),]
            beta_df = pd.concat([beta_df.loc[beta_df.condition_label==cti,] for cti in conditions_to_include])

        if condition_count_required is not None:
            if beta_df.shape[0]<condition_count_required:
                print("not enough conditions for subject "+ sl + ". Skipping this subject.")
                continue
        
        
        try: 
            #print(beta_df)
            bd = get_Brain_Data_betas_as_mvpa_for_sub(
                sl, beta_df,
                betaseries_path = first_level_path,
                events_in_design=beta_df.shape[0],
                #spatially_concatenate = concatenate_condition_labels,
                #mask = "beta" # '/projects/sanlab/shared/spm12/canonical/MNI152_T1_1mm_brain_mask.nii'
                #mask = '/projects/sanlab/shared/spm12/canonical/MNI152_T1_1mm_brain_mask.nii'
#                 mask=mask,
#                 masking_threshold=masking_threshold,
                **beta_processing_args
                
            )
            
            
            bd_dict[sl]=bd
        except BehavioralDataNotFoundForBrainDataException:
            print("couldn't get data for subject " + sl + " because there was no matching behavioral data")
    
            
    
    #now append into a concatenated brain data file
    
    Brain_Data_allsubs = mvpa2.datasets.vstack(list(bd_dict.values()),a=0)#list(bd_dict.values())[0]

    #now merge in supplementary data
    sa_index_cols = ['subject','wave']
    sa_index = sa_to_df(Brain_Data_allsubs.sa,sa_index_cols)
        
    if supplementary_df is not None:
        additional_cols = pd.merge(sa_index,supplementary_df,how='left',on=['subject','wave'])
        #the merge gets us the matched values for the shape of the sample attributes
        #now we need to put them back into the sample attributes
        for cname in additional_cols.columns:
            Brain_Data_allsubs.sa[cname] = additional_cols[cname]
            
    #easier to access version of the sa
    #Brain_Data_allsubs.a['get_sa_as_df'] = lambda: sa_to_df(my_test_data.sa)

    #dump
    out_filepath = (
        out_folder + 'mvpa_Dataset_' +
        #'betaseries_' +
        first_level_fileid + '_' + str(subj_count) + 'subs' + out_file_suffix + '.pkl'
    )
    print(out_filepath)

    with open(out_filepath, 'wb') as pkl_file:
        pickle.dump(Brain_Data_allsubs,pkl_file)
        
    return(Brain_Data_allsubs)

        
        

def get_Brain_Data_betas_as_mvpa_for_sub(
    sub_label,
    behavdesign_df,
    betaseries_path,
    events_in_design,
    mask = "beta",
    masking_threshold=0,
    spatially_concatenate=False,
    contrast_condition_1=None,
    contrast_condition_2=None
):
    """
    Collect a bunch of beta files for specified subjects based on a behavioral design file
    and return them
    """
    subj_behav_design = behavdesign_df[[s in sub_label for s in behavdesign_df.subject]]
    del(behavdesign_df)
    
    if (len(subj_behav_design)==0):
        raise BehavioralDataNotFoundForBrainDataException(
            "Found no behavioral data for subject " + sub_label + ". Please ensure the data exists - it is probably missing from the source.")
    elif len(subj_behav_design)!=events_in_design:
        raise Exception("For "+sub_label+", expected " +str(events_in_design) + " beta events but found " + str(len(subj_behav_design)))

    
    subject_dir = (
         betaseries_path + 'sub-' + sub_label + '/'
    )
        
    #set path for the betas

    #beta_path_sub049 = os.listdir(subject_dir)

    #iterate through the subject's beta files
    # TO DO
    #iterate through and check all the betas exist
    for betafile in subj_behav_design.beta:
        betafilepath = subject_dir + betafile
        
        #check if file exists
        if os.path.exists(betafilepath):
            print('.')
        else:
            print(betafilepath)
            raise Exception("beta " + betafile + ' does not exist at ' + betafilepath)

    print("...verified that all expected betas exist! Processing...")
    
    if mask=="beta":
        print("setting the mask to the first image in the series " + subj_behav_design.beta.iloc[0])

        with warnings.catch_warnings(record=True) as w:
            mask_path = subject_dir + subj_behav_design.beta.iloc[0]
            print(mask_path)
#             subj_mask_dataset = fmri_dataset(mask_path)
#             subj_mask_dataset.samples = np.isnan(subj_mask_dataset.samples)==False
#             mask = mvpa2.datasets.mri.map2nifti(subj_mask_dataset)
            mask = nib.load(mask_path)
            fdata = mask.get_fdata()
            fdata_notnan = np.isnan(fdata)==False
            mask=nib.nifti1.Nifti1Image(fdata_notnan,mask.affine,header=mask.header)
            from nilearn import plotting
            plotting.plot_stat_map(mask)
            #load the first image to be a mask


    #now need to ensure hte mask is in the right space

    
    if type(mask)==str:
        #import the mask

        mask = nib.load(mask)


    
    #now check to see if it's in the same space as the images.
    first_dataset_img = nib.load(subject_dir + subj_behav_design.beta.iloc[0])
    if mask.shape!=first_dataset_img.shape:
        print(first_dataset_img.affine)
        print(mask.affine)
        print(type(mask))
        mask = resample_img(mask,first_dataset_img.affine,target_shape = first_dataset_img.shape)
    
    
    #finally, binarize the mask
    mask_binarized = (mask.get_fdata()>masking_threshold).astype(float)
    print(np.sum(mask_binarized==0),end=", ")
    print(np.sum(mask_binarized>0),end=", ")
    
    #import them, but catch a very specific warning and don't show it because it gets annoying
    #https://docs.python.org/3/library/warnings.html
    with warnings.catch_warnings(record=True) as w:
        #print([subject_dir + b for b in subj_behav_design.beta])
        if (contrast_condition_1 is None) & (contrast_condition_2 is None):
            subj_data = fmri_dataset([subject_dir + b for b in subj_behav_design.beta],mask=mask_binarized)
            
            print(np.sum(subj_data.samples>0))

            #now we concatenate
            if spatially_concatenate:
                raise Exception("spatial concatenation not implemnted")
        elif (contrast_condition_1 is not None) & (contrast_condition_2 is not None):
            contrast_1_design = subj_behav_design[subj_behav_design.condition_label==contrast_condition_1]
            contrast_2_design = subj_behav_design[subj_behav_design.condition_label==contrast_condition_2]
            subj_data_1 = fmri_dataset([subject_dir + b for b in contrast_1_design.beta],mask=mask_binarized)
            subj_data_2 = fmri_dataset([subject_dir + b for b in contrast_2_design.beta],mask=mask_binarized)
            subj_data = subj_data_1
            subj_data.samples = subj_data_1.samples-subj_data_2.samples
        else:
            raise Exception("must have BOTH contrast_condition_1 and contrast_condition_2 or NEITHER")
            
        subj_data.samples[np.isnan(subj_data.samples)]=0
        print(subj_behav_design)
        #populate the sample attributes with the contents of the beta data frame
        #can't simply assign becasue we want to resepct the standard format used in pymvpa
        if (contrast_condition_1 is not None) & (contrast_condition_2 is not None):
            #process wehre we're combining two images into one
            for cname in subj_behav_design.columns:
                col_data = np.unique(subj_behav_design[cname])
                if len(col_data)==1:
                    subj_data.sa[cname] = col_data
                else:
                    #if type(col_data[0])!=str: col_data = [str(x) for x in col_data]
                    #still more than one item for some reason; let's string concatenate
                    subj_data.sa[cname] = [", ".join([str(x) for x in subj_behav_design[cname]])]
                    #subj_data.sa[cname] = [col_data]
                    
            print(subj_data.sa)
        else:
            #regular process
            for cname in subj_behav_design.columns:
                subj_data.sa[cname] = list(subj_behav_design[cname])

    return(subj_data)

<>:69: DeprecationWarning: invalid escape sequence \(
<>:69: DeprecationWarning: invalid escape sequence \(
/tmp/ipykernel_24573/2285069596.py:69: DeprecationWarning: invalid escape sequence \(
  condition_label = re.search('Sn\(1\)\s(.*)\*bf\(1\)',b_description)
